# Answering Business Questions using SQL

At first we will need to create some helper functions - for quick access to the database and for running a query.

In [2]:
import sqlite3, pandas as pd

def run_query(sql_query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(sql_query, conn)
    
def run_command(sql_command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(sql_command)
        
def show_tables():
    q = """
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    """
    return run_query(q)

show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


# Sales by genre

We need to decide **which** of the following albums **we should purchase** for our store, namely, for an **audience** from the **USA**:

| Artist Name | Genre |
|-------------|-------|
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

For this task we need to find the number of tracks sold for each genre in USA.

In [3]:
genre_sales_usa_query = """

WITH

customers_usa AS
(
SELECT customer_id
FROM customer
WHERE country = 'USA'
),

tracks_quantity AS
(
SELECT
    il.track_id,
    il.invoice_line_id
FROM customers_usa AS cu
INNER JOIN invoice AS i ON i.customer_id = cu.customer_id
INNER JOIN invoice_line AS il ON il.invoice_id = i.invoice_id
),

genre_tracks_quantity AS
(
SELECT
    g.name AS genre,
    COUNT(tq.invoice_line_id) AS tracks_quantity
FROM tracks_quantity AS tq
INNER JOIN track AS t ON t.track_id = tq.track_id
INNER JOIN genre AS g ON g.genre_id = t.genre_id
GROUP BY g.name
)

SELECT
    genre,
    tracks_quantity,
    CAST(tracks_quantity AS FLOAT) / (
        SELECT SUM(tracks_quantity)
        FROM genre_tracks_quantity
        ) AS tracks_percantage
FROM genre_tracks_quantity
ORDER BY tracks_quantity DESC;

"""

run_query(genre_sales_usa_query)

,genre,tracks_quantity,tracks_percantage
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


Based on our findings, **we could recommend** the following artists, which tracks are worth purchasing for our store (in this order):

* Red Tone, punk
* Slim Jim Bites, blues
* Meteor and the Girls, pop

# Sales by employee

The next task is related to **employee sales**.  
Who sold the most tracks and do we see a connection between the number of sales  and other employee attributes?  
Let's look at employee sales, their title, who they report to, and their hire dates.

In [4]:
employee_sales_query = '''
WITH

sales_per_customer AS
    (
    SELECT
        SUM(i.total) AS total,
        c.support_rep_id,
        i.customer_id
    FROM invoice AS i
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    GROUP BY c.customer_id, c.support_rep_id
    ),

employee_support_agent AS
    (
    SELECT
        e2.first_name || ' ' || e2.last_name AS reports_to,
        e1.employee_id
    FROM employee AS e1
    LEFT JOIN employee AS e2 ON e2.employee_id = e1.reports_to
    )

SELECT
    e.first_name || ' ' || e.last_name AS employee_name,
    e.title,
    esa.reports_to,
    (JULIANDAY() - JULIANDAY(e.hire_date)) / 365.25 AS years_worked,
    SUM(spc.total) AS total_sales
FROM sales_per_customer AS spc
INNER JOIN employee AS e ON e.employee_id = spc.support_rep_id
INNER JOIN employee_support_agent AS esa ON esa.employee_id = e.employee_id
GROUP BY e.employee_id
ORDER BY total_sales DESC

;

'''

run_query(employee_sales_query)

,employee_name,title,reports_to,years_worked,total_sales
0,Jane Peacock,Sales Support Agent,Nancy Edwards,6.202129,1731.51
1,Margaret Park,Sales Support Agent,Nancy Edwards,6.114518,1584.00
2,Steve Johnson,Sales Support Agent,Nancy Edwards,5.657297,1393.92


Here we can only see some **correlation between sales and years worked**.  
If we increase the sales of the employee who is in third place ( Steve Johnson) in proportion to the years worked by the employee who is in first place, we get (1393.92 * 6.164042) / 5.619210 =) 1529.073. That's not the same as Jane Peacock's sales, but helps us to see the pattern.

# Sales by country

Next, we need to find the following **sales figures for each country**:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Where a country has only one customer we will add this country to the "Other" group.

In [5]:
sales_by_country_query = '''

WITH

total_customers AS
    (
    SELECT
        country,
        count,
        CASE
            WHEN count = 1 THEN 'Other'
            ELSE country
        END AS country_or_other,
        CASE
            WHEN count = 1 THEN 0
            ELSE 1
        END AS sort
    FROM
        (
        SELECT
            country,
            COUNT(customer_id) AS count
        FROM customer
        GROUP BY country
        )
    ),
    
indicators AS
    (
    SELECT
        tc.country_or_other,
        SUM(sales.total) AS total_sales,
        AVG(sales.total) AS average_order,
        SUM(sales.total) / COUNT(DISTINCT customer_id) AS average_sales_per_customer
    FROM 
        (
        SELECT
            c.customer_id,
            c.country,
            i.total
        FROM customer AS c
        LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
        )
        AS sales
    LEFT JOIN total_customers AS tc ON tc.country = sales.country
    GROUP BY country_or_other
    )

SELECT
    tc.country_or_other AS country,
    tc.total_customers,
    i.total_sales,
    i.average_sales_per_customer,
    i.average_order
FROM
    (
    SELECT
        SUM(count) AS total_customers,
        country_or_other,
        sort
    FROM total_customers
    GROUP BY country_or_other
    )
    AS tc
LEFT JOIN indicators AS i
    ON i.country_or_other = tc.country_or_other
ORDER BY tc.sort DESC, i.total_sales DESC

;

'''

run_query(sales_by_country_query)

,country,total_customers,total_sales,average_sales_per_customer,average_order
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.448571


We could see here several countries with potential for a marketing campaign:

* Czech Republic

* India

* Portugal

Each of these countries has only 2 customers and at the same time the highest average sales per customer.  
But still we can only advise small marketing campaign - because this sample is not big enough for representative results.

# Albums vs individual tracks

In order to understand should we continue buying full albums from record companies, we need to find the following:
* **how many invoices do we have with only an album purchase**, without any additional tracks
* and, in opposite, **how many invoices do we have with tracks added manually**

In [14]:
albums_vs_individual_tracks_query = '''

WITH

album_purchased_column AS
    (
    SELECT
        *,
        CASE
            WHEN
                (
                SELECT
                    track_id
                FROM invoice_line
                WHERE invoice_line.invoice_id = invoice.invoice_id

                EXCEPT

                SELECT
                    track_id
                FROM track
                WHERE
                    album_id =
                        (
                        SELECT
                            album_id
                        FROM track
                        WHERE track_id =
                            (
                            SELECT
                                track_id
                            FROM invoice_line
                            WHERE invoice_id = invoice.invoice_id
                            LIMIT 1
                            )
                        )

                ) IS NULL

            AND

                (
                SELECT
                    track.track_id
                FROM track
                WHERE
                    track.album_id =
                        (
                        SELECT
                            album_id
                        FROM track
                        WHERE track_id =
                            (
                            SELECT
                                track_id
                            FROM invoice_line
                            WHERE invoice_id = invoice.invoice_id
                            LIMIT 1
                            )
                        )

                EXCEPT

                SELECT
                    track_id
                FROM invoice_line
                WHERE invoice_line.invoice_id = invoice.invoice_id
                
                ) IS NULL

                THEN 'yes'
            ELSE 'no'
        END AS album_purchase
    FROM invoice
    )

SELECT
    album_purchase,
    number_of_invoices,
    CAST(number_of_invoices AS FLOAT)/(SELECT COUNT(invoice_id) FROM invoice)
        AS percantage_of_invoices
FROM
    (
    SELECT
        album_purchase,
        COUNT(album_purchase) AS number_of_invoices
    FROM album_purchased_column
    GROUP BY album_purchase
    )

;

'''

run_query(albums_vs_individual_tracks_query)

,album_purchase,number_of_invoices,percantage_of_invoices
0,no,500,0.814332
1,yes,114,0.185668


Of course, to be sure of the results, we need to understand the ratio of popular and unpopular songs in albums and the correlation of track purchase costs and sales volume. Thus, we need to know the track sales structure first to be sure that there are a sufficient amount of songs that don't sell well and that we could save on.  
Besides, we are not sure what the part of customers, who purchasing albums, will do - will they buy less or maybe they go to another store then?

For now, there is a significant part of "albums" people, so **our advice is to keep buying full versions of the albums** until we have more data.